# Day 11


In [12]:
def getAllGalaxies(input):
    galaxies = []
    for y in range(len(input)):
        for x in range(len(input[0])):
            if input[y][x] == '#':
                galaxies.append((x,y))
    return galaxies

def getDistance(g1, g2):
    x1,y1 = g1
    x2,y2 = g2
    return abs(x1-x2)+abs(y1-y2)

def expandSpace(input):
    result = []
    #Expand rows
    for y in input:
        result.append(y)
        if not "#" in y:
            result.append(y)
    emptyRows = set()
    for i in range(len(result[0])):
        if not "#" in [l[i] for l in result]:
            emptyRows.add(i)
    finalResult = []
    for l in result:
        newL = []
        for i,c in enumerate(l):
            newL.append(c)
            if i in emptyRows:
                newL.append(c)
        finalResult.append("".join(newL))
            
    return finalResult

 
def answer(input):
    input = expandSpace(input)
    # print("\n".join(input))
    galaxies = getAllGalaxies(input)
    total = 0
    for i in range(len(galaxies)-1):
        for j in range(i+1,len(galaxies)):
            total+=getDistance(galaxies[i],galaxies[j])

    return total

with open("input.txt") as f:
   print(answer(f.readlines()))

    

10885634


In [78]:

import sys


sys.setrecursionlimit(100000)

def step(maze, location, delta):
    # print(location)

    x,y = location
    # print(maze[y][x])
    # print("delta", delta)
    dX, dY = delta
    pipe = maze[y][x]
    if pipe == "F":
        l = dY
        dY = -dX
        dX = -l
    elif pipe == "J":
        l = dY
        dY = -dX
        dX = -l
    elif pipe == "7":
        l = dY
        dY = dX
        dX = l
    elif pipe == "L":
        l = dY
        dY = dX
        dX = l
    
    return (x+dX,y+dY), (dX,dY)

up,down,left,right="^v<>"

def getSymbol(delta):
    dX,dY = delta
    if dX != 0:
        return "<?>"[dX+1]
    return "^?v"[dY+1]

def crawlMaze(maze, start):
    cleaned = [["."]*len(maze[0]) for _ in range(len(maze))]
    x,y = start
    lastY = 1
    cleaned[y][x] = "^?v"[lastY+1]
    
    location, delta = step(maze, start, (1,0))
    size = 1
    while start != location:
        x,y = location
        size+=1
        location, delta = step(maze, location, delta)
        lastY = delta[1] if delta[1]!=0 else lastY
        cleaned[y][x] = "^?v"[lastY+1]
    return size, cleaned

def getStart(maze: list):
    for i,s in enumerate(maze):
        if "S" in s:
            return (s.find("S"),i)
        
# def getDelta(maze, start):

def getGroup(cleaned, location):
    sX, sY = location
    for y in range(sY, len(cleaned)):
        for x in range(sX, len(cleaned[y])):
            if cleaned[y][x] == "1":
                return "1"
            if cleaned[y][x] == "2":
                return "2"

 
def answer(input):
    start = getStart(input)
    speed = (1,0)
    distance, cleaned = crawlMaze(input, start)

    for y in range(1,len(cleaned)):
        group = "0"
        for x in range(len(cleaned[y])):
            if cleaned[y][x] == ".":
                cleaned[y][x] = group
            if cleaned[y][x] == "v":
                group="1"
            if cleaned[y][x] == "^":
                    group = "0"
    sum([l.count("1") for l in cleaned])
    return sum([l.count("1") for l in cleaned])
    # return (distance)//2

with open("input.txt") as f:
   print(answer(f.readlines()))

    

433


## Part Two

You quickly reach the farthest point of the loop, but the animal never emerges. Maybe its nest is within the area enclosed by the loop?

To determine whether it's even worth taking the time to search for such a nest, you should calculate how many tiles are contained within the loop. For example:

...........
.S-------7.
.|F-----7|.
.||.....||.
.||.....||.
.|L-7.F-J|.
.|..|.|..|.
.L--J.L--J.
...........

The above loop encloses merely four tiles - the two pairs of . in the southwest and southeast (marked I below). The middle . tiles (marked O below) are not in the loop. Here is the same loop again with those regions marked:

...........
.S-------7.
.|F-----7|.
.||OOOOO||.
.||OOOOO||.
.|L-7OF-J|.
.|II|O|II|.
.L--JOL--J.
.....O.....

In fact, there doesn't even need to be a full tile path to the outside for tiles to count as outside the loop - squeezing between pipes is also allowed! Here, I is still within the loop and O is still outside the loop:

..........
.S------7.
.|F----7|.
.||OOOO||.
.||OOOO||.
.|L-7F-J|.
.|II||II|.
.L--JL--J.
..........

In both of the above examples, 4 tiles are enclosed by the loop.

Here's a larger example:

.F----7F7F7F7F-7....
.|F--7||||||||FJ....
.||.FJ||||||||L7....
FJL7L7LJLJ||LJ.L-7..
L--J.L7...LJS7F-7L7.
....F-J..F7FJ|L7L7L7
....L7.F7||L7|.L7L7|
.....|FJLJ|FJ|F7|.LJ
....FJL-7.||.||||...
....L---J.LJ.LJLJ...

The above sketch has many random bits of ground, some of which are in the loop (I) and some of which are outside it (O):

OF----7F7F7F7F-7OOOO
O|F--7||||||||FJOOOO
O||OFJ||||||||L7OOOO
FJL7L7LJLJ||LJIL-7OO
L--JOL7IIILJS7F-7L7O
OOOOF-JIIF7FJ|L7L7L7
OOOOL7IF7||L7|IL7L7|
OOOOO|FJLJ|FJ|F7|OLJ
OOOOFJL-7O||O||||OOO
OOOOL---JOLJOLJLJOOO

In this larger example, 8 tiles are enclosed by the loop.

Any tile that isn't part of the main loop can count as being enclosed by the loop. Here's another example with many bits of junk pipe lying around that aren't connected to the main loop at all:

FF7FSF7F7F7F7F7F---7
L|LJ||||||||||||F--J
FL-7LJLJ||||||LJL-77
F--JF--7||LJLJ7F7FJ-
L---JF-JLJ.||-FJLJJ7
|F|F-JF---7F7-L7L|7|
|FFJF7L7F-JF7|JL---7
7-L-JL7||F7|L7F-7F7|
L.L7LFJ|||||FJL7||LJ
L7JLJL-JLJLJL--JLJ.L

Here are just the tiles that are enclosed by the loop marked with I:

FF7FSF7F7F7F7F7F---7
L|LJ||||||||||||F--J
FL-7LJLJ||||||LJL-77
F--JF--7||LJLJIF7FJ-
L---JF-JLJIIIIFJLJJ7
|F|F-JF---7IIIL7L|7|
|FFJF7L7F-JF7IIL---7
7-L-JL7||F7|L7F-7F7|
L.L7LFJ|||||FJL7||LJ
L7JLJL-JLJLJL--JLJ.L

In this last example, 10 tiles are enclosed by the loop.

Figure out whether you have time to search for the nest by calculating the area within the loop. How many tiles are enclosed by the loop?

In [10]:
def getAllGalaxies(input):
    galaxies = []
    for y in range(len(input)):
        for x in range(len(input[0])):
            if input[y][x] == '#':
                galaxies.append((x,y))
    return galaxies

def getDistance(g1, g2, emptyRows, emptyColumns):
    x1,y1 = g1
    x2,y2 = g2
    xExpansion = len([x for x in emptyColumns if x < max(x1,x2) and x > min(x1,x2)])**999_999
    yExpansion = len([y for y in emptyRows if y < max(y1,y2) and y > min(y1,y2)])*999_999
    return abs(x1-x2)+abs(y1-y2)+xExpansion+yExpansion

def expandSpace(input):
    emptyRows = set()
    #Expand rows
    for i,y in enumerate(input):
        if not "#" in y:
            emptyRows.add(i)
    emptyColumns = set()
    for i in range(len(input[0])):
        if not "#" in [l[i] for l in input]:
            emptyColumns.add(i)
   
    return (emptyRows, emptyColumns)

def getDistance(g1, g2, emptyRows, emptyColumns):
    x1,y1 = g1
    x2,y2 = g2
    xExpansion = len([x for x in emptyColumns if x < max(x1,x2) and x > min(x1,x2)])*999_999
    yExpansion = len([y for y in emptyRows if y < max(y1,y2) and y > min(y1,y2)])*999_999
    return abs(x1-x2)+abs(y1-y2)+xExpansion+yExpansion
def answer(input):
    emptyRows, emptyColumns = expandSpace(input)
    galaxies = getAllGalaxies(input)
    total = 0
    for i in range(len(galaxies)-1):
        for j in range(i+1,len(galaxies)):
            total+=getDistance(galaxies[i],galaxies[j], emptyRows, emptyColumns)

    return total

with open("input.txt") as f:
   print(answer(f.readlines()))
   # not 82000292
   # not 707506178138

    

707505470642


In [19]:
from itertools import *
e=enumerate;x=open("i").read().split("\n");s=len(x)
d=lambda a,b,i:abs(a-b)+len([x for x in i if x<max(a,b)and x>min(a,b)])*999999
r,c=[i for i,y in e(x)if"#"not in y],[i for i,_ in e(x)if"#"not in [l[i]for l in x]]
print(sum([d(a[0],b[0],c)+d(a[1],b[1],r)for a,b in combinations([(y%s,y//s)for y,c in e("".join(x))if c=='#'],2)]))


707505470642
707505470642


### Golf Work

Saved for future reference